## HMM

HMM (Hidden Markov Model) is a process of randomly generating observation sequences from hidden Markov chains, which is a classic probabilistic graphical model.

<img src="1.png" />

HMM is a probability model about time series, in which the sequence of states randomly generated by the hidden Markov chain is called the state sequence, and the sequence of states generated by each state is called the observation sequence.

HMM is determined by initial state probability vector $\pi$, state transition probability matrix $A$ and observation probability matrix $B$. Among these three, $\pi$ and $A$ determine the state sequence, $B$ determines the observation sequence. Therefore, HMM can be represented by a ternary notation: 
$$
\lambda = (A, B, \pi)
$$

Set $Q$ to be the set of $N$ possible states and $V$ to be the set of $M$ possible observations:
$$
Q=\{q_{1},q_{2},\cdots,q_{N}\}, \enspace V=\{v_{1},v_{2},\cdots,v_{M}\}
$$

$I$ is a state sequence with length $T$ and $O$ is a observation sequence with length $T$:
$$
I=(i_{1},i_{2},\cdots,i_{T}), \enspace O=(o_{1},o_{2},\cdots,o_{T})
$$

In [8]:
def rdistribution(dist): 
    r = np.random.rand()
    for ix, p in enumerate(dist):
        if r < p: 
            return ix
        r -= p

In [13]:
# generate observation sequence
def generate(N, M, pi, A, B, T):
    # generate the first state based on the initial probability
    i = rdistribution(pi)  
    # generate the first observation
    o = rdistribution(B[i])  
    observed_data = [o]
    # iteration for generating the remaining states and observations
    for _ in range(T-1):        
        i = rdistribution(A[i])
        o = rdistribution(B[i])
        observed_data.append(o)
    return observed_data

 $A$ is state transition matrix that can be expressed as:
$$
A=[a_{ij}]_{N \times N}
$$

$a_{ij}$ represents the probability of transition from state $q_{i}$ at time $t$ to state $q_{j}$ at time $t+1$:
$$
a_{ij}=P(i_{t+1}=q_{j}|i_{t}=q_{i}), \enspace i,j=1,2,\cdots,N
$$

$B$ is observation probability matrix that can be expressed as:
$$
B=[b_{j}(k)]_{N \times N}
$$

$b_{j}(k)$ represents the probability of generating observation $v_{k}$ from state $q_{j}$ at time $t$:
$$
\begin{aligned}
&b_{j}(k)=P(o_{t}=v_{k}|i_{t}=q_{j})\\
&k=1,2,\cdots,M; \enspace j=1,2,\cdots,M
\end{aligned}
$$

In [4]:
# initial state probability vector
pi = np.array([0.25, 0.25, 0.25, 0.25])
# state transition probability matrix
A = np.array([
    [0,  1,  0, 0],
    [0.4, 0, 0.6, 0],
    [0, 0.4, 0, 0.6],
[0, 0, 0.5, 0.5]])
# observation probability matrix
B = np.array([
    [0.5, 0.5],
    [0.6, 0.4],
    [0.2, 0.8],
    [0.3, 0.7]])
# number of state
N = 4
# number of observation
M = 2

There are three basic problem in HMM: probability estimation, learning problem and prediction problem.

### Estimation algorithm in HMM

The probability estimation problem in HMM is calculating $P(O|\lambda)$, which is the probability of observation sequence $O$ appearing in the model $\lambda$. And the probability estimation method of HMM is forward-backward algorithm. 

HMM define the forward probability as follow: the probability of observation sequence $o_{1},o_{2},\cdots,o_{t}$ that are in the state of $q_{i}$ at time $t$ under the given model $\lambda$
$$
\alpha _{t}(i)=P(o_{1},o_{2},\cdots,o_{t},i_{t}=q_{i}|\lambda)
$$

From the definition above, the forward probability is a joint probability, and forward probability $\alpha _{t}(i)$ and observation sequence probability $P(O|\lambda)$ can be calculated recursively.With the input of HMM model $\lambda$ and observation sequence $O$, output of observation sequence probability $P(O|\lambda)$, the forward algorithm is expressed as below:

• Value initialization: $\alpha _{1}(i)=\pi_{i}b_{i}(o_{1}), \enspace i=1,2,\cdots,N$

• Recursion: $\alpha_{i+1}(i)=\left[\sum_{j=1}^{N} \alpha_{t}(j) a_{j i}\right] b_{i}\left(o_{t+1}\right), \quad i=1,2, \ldots, N$ for $t=1,2,\cdots,T-1$

• Termination: $P(O|\lambda)=\sum^{N}_{i=1} \alpha_ {T}(i)$

The specific derivation process of the forward probability algorithm is given below. According to the definition of forward probability, the initial value can be deduced as:
$$
\begin{aligned}
\alpha_{1}(i)&=P(o_{1},i_{1}=q_{i}|\lambda) \\
&=P(i_{1}=q_{i}|\lambda)P(o_{1}|i_{1}=q_{i},\lambda) \\
&=\pi_{i}b_{i}(o_{1})
\end{aligned}
$$

$b_{i}(x)$ indicates the probability of generating state $q_{i}$ in observation, and assuming that observed data $o_{t}=v_{j}$ at time $t$:
$$
b_{i}(o_{t})=b_{i}(o_{t}=v_{j})=P(o_{t}=v_{j}|i_{t}=q_{i})=b_{ij}
$$

Based on the definition of forward probability:
$$
\begin{aligned}
\alpha _{T}(i)&=P(o_{1},o_{2},\cdots,o_{T}, i_{T}=q_{i}|\lambda) \\
&=P(O, i_{T}=q_{i}|\lambda)
\end{aligned}
$$

According to the above formula, traversing the value of $i_{T}$ and summing them, the marginal probability of $O$ can be obtained:
$$
\sum ^{N}_{i} \alpha_{T}(i)=\sum ^{N}_{i} P(O, i_{T}=q_{i}|\lambda) = P(O|\lambda)
$$

Let $i_{t}=q_{i}$, there is:
$$
\begin{aligned}
\alpha_{t+1}(j)
&=\sum_{i=1}^{N} P\left(o_{1}, o_{2}, \ldots, o_{t+1}, i_{t}=q_{i}, i_{t+1}=q_{j} \mid \lambda\right) \\
&=\sum_{i=1}^{N} P\left(o_{t+1}, o_{1}, \ldots, o_{t}, i_{t}=q_{i}, i_{t+1}=q_{j} \mid \lambda\right) P\left(o_{1}, o_{2}, \ldots, o_{t}, i_{t}=q_{i}, i_{t+1}=q_{j} \mid \lambda\right)
\end{aligned}
$$

In the derivation of the above formula, the first term can be simplified to:
$$
\begin{aligned}
&P\left(o_{1}, o_{2}, \ldots, o_{t+1}, i_{t}=q_{i}, i_{t+1}=q_{j} \mid \lambda\right) \\
=& P(o_{t+1}|i_{t+1}=q_{j}) \\
=& b_{j}(o_{t+1})
\end{aligned}
$$

According to the homogeneous Markov assumption the second term can be simplified as:
$$
\begin{aligned}
&P\left(o_{t+1}, o_{1}, \ldots, o_{t}, i_{t}=q_{i}, i_{t+1}=q_{j} \mid \lambda\right) \\ 
&P\left(o_{1}, o_{2}, \ldots, o_{t}, i_{t}=q_{i}, i_{t+1}=q_{j} \mid \lambda\right) \\
=&P(i_{t+1}=q_{j}|i_{t}=q_{i})P(o_{1},\cdots,o_{t}, i_{t}=q_{i}|\lambda) \\
=& a_{ij} \alpha_{t}(i)
\end{aligned}
$$

Assuming that $\alpha_{t}(1), \cdots, \alpha_{t}(N) $ is known, and combining with the simplified results above, there is:
$$
\begin{aligned}
\alpha_{t+1}(j)&=P(o_{1},o_{2},\cdots,o_{t+1},i_{t+1}=q_{j}|\lambda) \\
&= \sum^{N}_{i=1}a_{ij}b_{j}(o_{t+1})\alpha_{t}(i)
\end{aligned}
$$

The backward probability algorithm can be derived in the same way.

In [16]:
# forward algorithm to calculate condition probability
def prob_calc(O):
    '''
    input：O(observation sequence)
    output：alpha.sum()(condition probability)
    '''
    # initial value
    alpha = pi * B[:, O[0]]
    # recursion
    for o in O[1:]:
        alpha_next = np.empty(4)
        for j in range(4):
            alpha_next[j] = np.sum(A[:,j] * alpha * B[j,o])
        alpha = alpha_next
    return alpha.sum()

### Learning algorithm in HMM

The learning problem in HMM is that, how to estimate the parameters in HMM model $\lambda=(A,B, \pi)$ to maximize the observation sequence probability $P(O|\lambda)$ when the observation sequence $O=(o_{1}, o_{2}, \cdots, o_{T})$ is known.

If both observation sequence and corresponding state sequence are included in the training data, MLE can be used directly to estimate the model parameters. But in most cases, only observation sequence is in the training data. In this situation, the observation sequence is the observed data $O$ and the state sequence is the unobservable hidden data $I$, so the HMM model is actually a probability model with hidden variables.
$$
P(O|\lambda)=\sum _{I}P(O|I, \lambda)P(I|\lambda)
$$

Probabilistic models with hidden variables can be iteratively solved by the EM algorithm. Applying EM algorithm to solve HMM model parameters is also called the Baum-Welch algorithm. The process of solving HMM model parameters based on the Baum-Welch algorithm is as follows:

(1) Determining the log-likelihood function for complete data: set the observed data as $O=(o_{1},o_{2},\cdots,o_{T})$, hidden data as $I=(i_{1},i_{2},\cdots,i_{T})$, complete data as $(O,I)=(o_{1},o_{2},\cdots,o_{T},i_{1},i_{2},\cdots,i_{T})$, and log-likelihood function for complete data as $\log P(O,I|\lambda)$

(2) E step in EM algorithm: solve Q function $Q(\lambda, \bar{\lambda})$, $\bar{\lambda}$ is the current estimation for HMM parameters, ${\lambda}$ is to maximize HMM parameters
$$
Q(\lambda, \bar{\lambda}) = \sum_{I} \log P(O,I|\lambda)P(O,I|\bar{\lambda})  \\
P(O, I \mid \lambda)=\pi_{i_{1}} b_{i_{1}}\left(o_{1}\right) a_{i_{1} i_{2}} b_{i_{2}}\left(o_{2}\right) \ldots a_{i_{T-1} i_{T}} b_{i_{T}}\left(o_{T}\right)
$$

(3) M step in EM algorithm: maximize Q function $Q(\lambda, \bar{\lambda})$ to obtain model parameters $A, B, \pi$

### Prediction algorithm in HMM

The prediction problem of HMM is finding the most likely corresponding state sequence $I=(i_{1},i_{2},\cdots,i_{T})$ (with largest condition probability $P(I|O)$) when the model $\lambda=(A,B,\pi)$ and observation sequence $O=(o_{1},o_{2},\cdots,o_{T})$ are known.

The prediction algorithm of HMM is a problem of solving the optimal path that it can be solved by Viterbi algorithm. The HMM optimal path solution process based on Viterbi algorithm is as follows:

(1) Initialization: 
$$
\delta_{1}(i)=\pi_{i} b_{i}\left(o_{1}\right), \enspace \Psi_{1}(i)=0, \enspace i=1,2, \ldots, N
$$

(2) Recursion: for $t=2,3,\cdots,T$
$$
\begin{aligned}
&\delta_{t}(i)=\max _{1<=j<=N}\left[\delta_{t-1}(j) a_{j i}\right] b_{i}\left(o_{t}\right) \\
&\Psi_{t}(i)=\arg \max _{1<=j<=N}\left[\delta_{t-1}(j) a_{j i}\right], \quad i=1,2, \ldots, N
\end{aligned}
$$

(3) Termination: 
$$
P^{*}=\max _{1<=j<=N} \delta_{T}(i) i_{T}^{*}=\arg \max _{1<=j<=N}\left[\delta_{T}(i)\right]
$$

(4) Optimal path backtracking: for $t=T-1, T-2, \cdots, 1$
$$
i^{*}_{t}= \Psi_{t+1}(i^{*}_{t+1})
$$

In [19]:
def viterbi_decode(O):
    '''
    input：O(observation sequence)
    output：path(optimal hidden state path)
    '''    
    # sequence length and initial observation
    T, o = len(O), O[0]
    # initialize delta 
    delta = pi * B[:, o]
    # initialize varphi
    varphi = np.zeros((T, 4), dtype=int)
    path = [0] * T
    # recursion
    for i in range(1, T):
        delta = delta.reshape(-1, 1)     
        tmp = delta * A
        varphi[i, :] = np.argmax(tmp, axis=0)
        delta = np.max(tmp, axis=0) * B[:, O[i]]
    # termination
    path[-1] = np.argmax(delta)
    # optimal path backtracking
    for i in range(T-1, 0, -1):
        path[i-1] = varphi[i, path[i]]
    return path

In [3]:
import numpy as np

In [20]:
hmm = generate(N, M, pi, A, B, 5)
print(hmm)
print(prob_calc(hmm))
print(viterbi_decode(hmm))

[0, 1, 1, 1, 1]
0.06247567374999999
[1, 2, 3, 2, 3]
